# Practical Exam: Hotel Operations

LuxurStay Hotels is a major, international chain of hotels. They offer hotels for both business and leisure travellers in major cities across the world. The chain prides themselves on the level of customer service that they offer. 

However, the management has been receiving complaints about slow room service in some hotel branches. As these complaints are impacting the customer satisfaction rates, it has become a serious issue. Recent data shows that customer satisfaction has dropped from the 4.5 rating that they expect. 

You are working with the Head of Operations to identify possible causes and hotel branches with the worst problems. 

## Data

The following schema diagram shows the tables available. You have only been provided with data where customers provided a feedback rating.

![hotel_operations](hotel_operations.png)

# Task 1

Before you can start any analysis, you need to confirm that the data is accurate and reflects what you expect to see. 

It is known that there are some issues with the `branch` table, and the data team have provided the following data description. 

Write a query to return data matching this description. You must match all column names and description criteria.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|id | Nominal. The unique identifier of the hotel. </br>Missing values are not possible due to the database structure.|
| location | Nominal. The location of the particular hotel. One of four possible values, 'EMEA', 'NA', 'LATAM' and 'APAC'. </br>Missing values should be replaced with “Unknown”. |
| total_rooms | Discrete. The total number of rooms in the hotel. Must be a positive integer between 1 and 400. </br>Missing values should be replaced with the default number of rooms, 100. |
| staff_count | Discrete. The number of staff employeed in the hotel service department. </br>Missing values should be replaced with the total_rooms multiplied by 1.5. |
| opening_date | Discrete. The year in which the hotel opened. This can be any value between 2000 and 2023. </br>Missing values should be replaced with 2023. |
| target_guests | Nominal. The primary type of guest that is expected to use the hotel. Can be one of 'Leisure' or 'Business'. </br>Missing values should be replaced with 'Leisure'. |

In [87]:
-- Write your query for task 1 in this cell
SELECT id,
	COALESCE(location, 'Unknown') AS location, 
	COALESCE(total_rooms, 100) AS total_rooms, 
	COALESCE(staff_count, total_rooms * 1.5) AS staff_count, 
	REPLACE(opening_date, '-', '2023') AS opening_date, 
	REPLACE(REPLACE(target_guests, 'B.','Business'), 'Busniess', 'Business') AS target_guests
	
FROM public.branch




In [56]:
SELECT * FROM public.branch

,id,location,total_rooms,staff_count,opening_date,target_guests
0,1,LATAM,168.0,178,2017,Business
1,2,APAC,154.0,82,2010,Leisure
2,3,APAC,212.0,467,2003,Leisure
3,4,APAC,230.0,387,-,Business
4,5,APAC,292.0,293,2002,Business
...,...,...,...,...,...,...
95,96,APAC,237.0,257,2000,Business
96,97,APAC,107.0,169,2005,Business
97,98,EMEA,196.0,126,2002,Leisure
98,99,APAC,242.0,251,2021,Business


# Task 2

The Head of Operations wants to know whether there is a difference in time taken to respond to a customer request in each hotel. They already know that different services take different lengths of time. 

Calculate the average and maximum duration for each branch and service. Your output should include the columns `service_id`, `branch_id`, `avg_time_taken` and `max_time_taken`. Values should be rounded to two decimal places where appropriate. 

In [57]:
-- Write your query for task 2 in this cell
SELECT service_id, branch_id, ROUND(AVG(time_taken), 2) AS avg_time_taken, MAX(time_taken) AS max_time_taken
FROM public.request
GROUP BY service_id, branch_id
ORDER BY branch_id, service_id ASC;


,service_id,branch_id,avg_time_taken,max_time_taken
0,1,1,2.44,12
1,2,1,12.73,15
2,3,1,7.00,7
3,1,2,2.14,6
4,2,2,13.46,15
...,...,...,...,...
380,3,99,6.97,8
381,4,99,9.13,13
382,2,100,14.00,14
383,3,100,6.89,8


In [58]:
SELECT * FROM public.request

,id,service_id,branch_id,time_taken,request_time,rating
0,1,3,92,7,Day-Off-Peak,4
1,2,1,43,2,Day-Off-Peak,4
2,3,2,63,13,Day-Peak,4
3,4,3,89,7,Day-Peak,5
4,5,4,100,9,Day-Peak,4
...,...,...,...,...,...,...
17677,17678,3,68,7,Day-Peak,5
17678,17679,4,67,9,Day-Off-Peak,4
17679,17680,1,9,4,Day-Peak,3
17680,17681,1,23,2,Night,5


# Task 3

The management team want to target improvements in `Meal` and `Laundry` service in Europe (`EMEA`) and Latin America (`LATAM`). 

Write a query to return the `description` of the service, the `id` and `location` of the branch, the id of the request as `request_id` and the `rating` for the services and locations of interest to the management team. 

Use the original branch table, not the output of task 1. 

In [89]:
-- Write your query for task 3 in this cell
SELECT service.description, r.branch_id, branch.location, r.id as request_id, r.rating
FROM public.request as r

INNER JOIN (
	SELECT id, location
	FROM public.branch 
	WHERE location IN ('EMEA', 'LATAM')
) AS branch ON branch.id = r.branch_id

INNER JOIN (
	SELECT id, description
	FROM public.service 
	WHERE public.service.description IN ('Meal', 'Laundry')
)AS service ON service.id = r.service_id

ORDER BY branch_id DESC, request_id 


In [60]:
SELECT * FROM public.service

,id,description
0,1,Meal
1,2,Laundry
2,3,Cleaning
3,4,Other


# Task 4

So that you can take a more detailed look at the lowest performing hotels, you want to get service and branch information where the average rating for the branch and service combination is lower than 4.5 - the target set by management.  

Your query should return the `service_id` and `branch_id`, and the average rating (`avg_rating`), rounded to 2 decimal places.

In [74]:
-- Write your query for task 4 in this cell
SELECT service_id, branch_id, ROUND(AVG(rating),2) AS avg_rating
FROM public.request
GROUP BY service_id, branch_id
HAVING ROUND(AVG(rating),2) < 4.5
ORDER BY branch_id, service_id
